# Notebook #1: Data Extraction
## Importing tabular data onto Rhino with SQL queries
In this notebook, you'll use SQL to query from an external database (such as a health system's clinical data warehouse) and import the results of those queries onto the Rhino Federated Computing Platform.

### Import the Rhino Health Python library
The code below imports various classes and functions from the `rhino_health` library, which is a custom library designed to interact with the Rhino Federated Computing Platform. More information about the SDK can be found on our [Official SDK Documentation](https://rhinohealth.github.io/rhino_sdk_docs/html/autoapi/index.html) and on our [PyPI Repository Page](https://pypi.org/project/rhino-health/) 

In [1]:
# pip install --upgrade rhino_health

In [2]:
import getpass
from pprint import pprint
import rhino_health as rh
from rhino_health.lib.endpoints.dataset.dataset_dataclass import DatasetCreateInput
from rhino_health.lib.endpoints.data_schema.data_schema_dataclass import DataSchemaCreateInput
from rhino_health.lib.endpoints.project.project_dataclass import ProjectCreateInput
from rhino_health.lib.endpoints.sql_query.sql_query_dataclass import (SQLQueryImportInput,SQLQueryInput,SQLServerTypes,ConnectionDetails)
from rhino_health.lib.endpoints.code_object.code_object_dataclass import (CodeObject,CodeObjectCreateInput,CodeObjectRunInput,CodeTypes)

/Users/drewafromsky/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Authenticate to the Rhino FCP
The `RhinoSession` class in the `rhino_health` library is a comprehensive interface for interacting with various endpoints in the Rhino Health ecosystem. It offers direct access to multiple specialized endpoints, including AI models, cohorts, data schemas, model results, projects, and workgroups, facilitating a wide range of operations in healthcare data management and analysis. The class also supports features like two-factor authentication and user switching, enhancing security and flexibility in handling different user sessions and workflows within the Rhino Health platform.

In [3]:
my_username = "drew@rhinohealth.com"

print("Logging In")
session = rh.login(
    username=my_username, 
    password=getpass.getpass(), 
    show_traceback=True,
    rhino_api_url='https://dev.rhinohealth.com/api/'
)
print("Logged In")

Logging In


 ········


Logged In


### Identify the desired project in the Rhino UI.
Before completing this step using the Python SDK, create a project on the Rhino web platform. Once the project has been created, copy the UID from the project you just created in the UI by navigating to the homepage, pressing on the three-vertical dot button in your project's square, and then selecting the button Copy UID.


![image info](https://drive.google.com/uc?export=view&id=1QjVkMmHTCdCpQF3qysScJr8tiSXtecOz)

In [4]:
project_name = "Federated Datasets and Predictive Modeling (Setup Project + Collab) - drew@"

for project in session.project.get_projects():
    if project.name == project_name:
        print(project.name,"\n")
        print(project,"\n")
        project_uid = project.uid

Federated Datasets and Predictive Modeling (Setup Project + Collab) - drew@ 

Project session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040> creator_uid='e50f2461-3d7a-4fe0-b457-b2fe45d05532' created_at='2024-02-29T16:32:14.440680Z' name='Federated Datasets and Predictive Modeling (Setup Project + Collab) - drew@' description='Example Project for end-to-end Federated Modeling (project creation,                  add collaborator (invite + accept), Data Ext/Eng, Stat Analysis, training, and evaluation' type='Validation' primary_workgroup_uid='97d7456b-baef-4d88-bfb1-afd4975eaff2' permissions=None uid='1752fb63-c36c-4221-85a4-f979c5e921bb' slack_channel='' collaborating_workgroup_uids=['97d7456b-baef-4d88-bfb1-afd4975eaff2', '33cf1db0-de14-472a-8dcb-8d83de22d946', '48cb366f-b05f-4ca2-8e1d-6dfc336cd344'] 



In [6]:
project_uid

'1752fb63-c36c-4221-85a4-f979c5e921bb'

In [5]:
primary_workgroup_uid = session.current_user.primary_workgroup.uid
print(primary_workgroup_uid)

97d7456b-baef-4d88-bfb1-afd4975eaff2


In [8]:
session.project.get_collaborating_workgroups(project_uid)

[Workgroup(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040>, uid='97d7456b-baef-4d88-bfb1-afd4975eaff2', name='Rhino Health Test', org_name='Rhino Health Test'),
 Workgroup(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040>, uid='33cf1db0-de14-472a-8dcb-8d83de22d946', name='rhino-sandbox-aidev', org_name='Rhino Sandbox'),
 Workgroup(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040>, uid='48cb366f-b05f-4ca2-8e1d-6dfc336cd344', name='rhino-sandbox-hco', org_name='Rhino Sandbox')]

In [10]:
session.project.get_collaborating_workgroups(project_uid)[0].uid

'97d7456b-baef-4d88-bfb1-afd4975eaff2'

In [15]:
workgroup_uid = session.project.get_collaborating_workgroups(project_uid)[0].uid
print(workgroup_uid)

97d7456b-baef-4d88-bfb1-afd4975eaff2


In [11]:
for project in session.project.get_projects():
    print(project,"\n")

Project session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040> creator_uid='e50f2461-3d7a-4fe0-b457-b2fe45d05532' created_at='2024-02-29T16:32:14.440680Z' name='Federated Datasets and Predictive Modeling (Setup Project + Collab) - drew@' description='Example Project for end-to-end Federated Modeling (project creation,                  add collaborator (invite + accept), Data Ext/Eng, Stat Analysis, training, and evaluation' type='Validation' primary_workgroup_uid='97d7456b-baef-4d88-bfb1-afd4975eaff2' permissions=None uid='1752fb63-c36c-4221-85a4-f979c5e921bb' slack_channel='' collaborating_workgroup_uids=['97d7456b-baef-4d88-bfb1-afd4975eaff2', '33cf1db0-de14-472a-8dcb-8d83de22d946', '48cb366f-b05f-4ca2-8e1d-6dfc336cd344'] 

Project session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040> creator_uid='e50f2461-3d7a-4fe0-b457-b2fe45d05532' created_at='2024-02-22T18:51:38.729265Z' name='Federated Datasets and Predictive\xa0Modeling (Sandbox) - dre

### Connection Setup

- The `rhino_health.lib.endpoints.sql_query.sql_query_dataclass` module in the Rhino Health library provides classes to handle SQL queries against external databases and import data into the Rhino Federated Computing Platform. 
- `SQLQueryInput` for specifying parameters of a SQL query
- `SQLQueryImportInput` for importing a cohort from an external SQL database query
- `SQLQuery`: a class representing an executed SQL query
- `QueryResultStatus` define the status of query results
- `SQLServerTypes` define supported SQL server types
- `ConnectionDetails` class specifies connection details for an external SQL database.

- Rhino's SQL classes [here](https://rhinohealth.github.io/rhino_sdk_docs/html/autoapi/rhino_health/lib/endpoints/sql_query/index.html).
- When specifying the connection details, ensure that you provide the server_type using the approved SQLServerTypes enum. This step ensures that your server is supported and compatible with the querying process.

In [18]:
sql_db_user = "rhino" # Replace this with your DB username (make sure the user has read-only permissions to the DB).
external_server_url = "ext-hospital-data.covi47dnmpiy.us-east-1.rds.amazonaws.com:5432" # Replace this with url + port of the SQL DB you want to query (ie "{url}:{port}").
db_name = "hospital_data" # Replace this with your DB name.

connection_details = ConnectionDetails(
    server_user=sql_db_user,
    password=getpass.getpass(),    
    server_type=SQLServerTypes.POSTGRESQL, # Replace POSTGRESQL with the relevant type of your sql server (See docs for all supported types).
    server_url=external_server_url,
    db_name=db_name
)

 ········


In [ ]:
connection_details

### Writing SQL queries against the DB
Using the `SQLQueryImportInput` function will allow us to query an external relational database and import the results of the query as a cohort. A Cohort is a central concept on the Rhino platform; to learn more, please navigate to this [link](https://docs.rhinohealth.com/hc/en-us/articles/12384748397213-What-is-a-Cohort-)

Executing the `SQLQueryImportInput` function requires a few arguments:
- `cohort_name` (str): Name for the cohort you are creating.
- `is_data_deidentified (bool)`: Indicates if the data in the query is deidentified for privacy reasons.
- `connection_details (ConnectionDetails)`: Details like URL, user, and password to connect to the SQL server.
- `data_schema_uid (Optional[str])`: The unique identifier for the data schema in the context of the query.
- `timeout_seconds (int)`: Time limit in seconds for the query execution.
- `project_uid (str)`: Unique identifier for the project context of the query.
- `workgroup_uid (str)`: Unique identifier for the workgroup context of the query.
- `sql_query (str)`: The actual SQL query to be run.

#### Table 1: Patient Admission Data
Our first query will retrieve patient demographics and associated clinical codes from inpatient admissions for patients with chest x-rays (see the WHERE clause, where we identify a selection of chest x-rays in the MIMIC v4 database).

In [20]:
query_demo="""
SELECT distinct
      pat.subject_id
    , adm.hadm_id
    , pat.anchor_age + (EXTRACT(YEAR FROM adm.admittime) - pat.anchor_year) AS age
    , pat.gender
    , adm.insurance
    , adm.admission_type
    ,adm.admission_location
    ,adm.discharge_location
    ,adm.language
    ,adm.marital_status
    , adm.race
    , icd.icd_code as diagnosis_code
    ,proc.icd_code as procedure_code
FROM mimiciv_hosp.admissions adm
LEFT JOIN mimiciv_hosp.patients pat
ON pat.subject_id = adm.subject_id
LEFT JOIN mimiciv_hosp.diagnoses_icd icd
ON adm.subject_id = icd.subject_id
AND adm.hadm_id = icd.hadm_id
LEFT JOIN mimiciv_hosp.procedures_icd proc
ON adm.subject_id = proc.subject_id
AND adm.hadm_id = proc.hadm_id
LEFT JOIN mimiciv_cxr.study_list study
ON adm.subject_id =study.subject_id
WHERE study.study_id in(57375967,50771383,54205396,53186264,54684191,50531538,59965534,51029426,50336039,51967845,53970869,54898709,57084339,58117097,58509443,
58555910,58733084,59289932,51449744,55590752,56617354,57652741,50230446,50252971,50702026,50789010,51737583,54855307,56421164,58996402,59565087,52358194,
51732447,53447201,54136122,58882809,54076811,53022275,58261299,57661212,52821744,53831730,56167317,57754443,50548939,55758528,58974095,51613820,58898689,55328702,
52654671,53468612,55928380,53461983,58400857,57874958,58971884,59558528,51497652,53161617,54277770,59484629,51212589,58414548,53818182,58812027,59585309,53534710,
58748017,53445324,58890389,53977911,57107380,58184428,53522120,55014265)
"""
import_run_params = SQLQueryImportInput(
    session = session,
    project = project_uid, # The project/workgroup will be used to validate permissions (including and k_anonymization value)
    workgroup = workgroup_uid,
    connection_details = connection_details,
    timeout_seconds = 1200,
    is_data_deidentified = True,
    dataset_name='mimic_ehr_demo_dev',
    data_schema=None, # Auto-Generating the Output Data Schema for the Cohort
    sql_query = query_demo
)

response = session.sql_query.import_dataset_from_sql_query(import_run_params)

Waiting for SQL query to complete (0 hours 0 minutes and a second)
Waiting for SQL query to complete (0 hours 0 minutes and 12 seconds)
Run finished successfully


In [21]:
response

SQLQuery(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040>, creator_uid='e50f2461-3d7a-4fe0-b457-b2fe45d05532', created_at='Mar 04, 2024 07:56:16PM', project_uid='1752fb63-c36c-4221-85a4-f979c5e921bb', workgroup_uid='97d7456b-baef-4d88-bfb1-afd4975eaff2', sql_query='SELECT distinct\n      pat.subject_id\n    , adm.hadm_id\n    , pat.anchor_age + (EXTRACT(YEAR FROM adm.admittime) - pat.anchor_year) AS age\n    , pat.gender\n    , adm.insurance\n    , adm.admission_type\n    ,adm.admission_location\n    ,adm.discharge_location\n    ,adm.language\n    ,adm.marital_status\n    , adm.race\n    , icd.icd_code as diagnosis_code\n    ,proc.icd_code as procedure_code\nFROM mimiciv_hosp.admissions adm\nLEFT JOIN mimiciv_hosp.patients pat\nON pat.subject_id = adm.subject_id\nLEFT JOIN mimiciv_hosp.diagnoses_icd icd\nON adm.subject_id = icd.subject_id\nAND adm.hadm_id = icd.hadm_id\nLEFT JOIN mimiciv_hosp.procedures_icd proc\nON adm.subject_id = proc.subject_id\nAND adm.h

#### Table 2: EHR Observations
Our second query will retrieve observations from our clinical information system, including patient BMI, height, weight, and diastolic and systolic blood pressure.

In [22]:
query_obs = """
SELECT
   omr.subject_id,
   omr.chartdate,
   omr.result_name,
   max(omr.result_value) as result
FROM mimiciv_hosp.omr omr
LEFT JOIN mimiciv_cxr.study_list study
ON omr.subject_id =study.subject_id
WHERE study.study_id in (57375967,50771383,54205396,53186264,54684191,50531538,59965534,51029426,50336039,51967845,53970869,54898709,57084339,58117097,58509443,58555910,58733084,59289932,51449744,55590752,56617354,57652741,50230446,50252971,50702026,50789010,51737583,54855307,56421164,58996402,59565087,52358194,51732447,53447201,54136122,58882809,54076811,53022275,58261299,57661212,52821744,53831730,56167317,57754443,50548939,55758528,58974095,51613820,58898689,55328702,52654671,53468612,55928380,53461983,58400857,57874958,58971884,59558528,51497652,53161617,54277770,59484629,51212589,58414548,53818182,58812027,59585309,53534710,58748017,53445324,58890389,53977911,57107380,58184428,53522120,55014265)
GROUP BY omr.subject_id, omr.chartdate, omr.result_name
"""

import_run_params = SQLQueryImportInput(
    session = session,
    project = project_uid, # The project/workgroup will be used to validate permissions (including and k_anonymization value)
    workgroup = workgroup_uid,
    connection_details = connection_details,
    timeout_seconds = 1200,
    is_data_deidentified = True,
    dataset_name='mimic_ehr_obs_dev',
    data_schema=None, # Auto-Generating the Output Data Schema for the Cohort
    sql_query = query_obs
)

response = session.sql_query.import_dataset_from_sql_query(import_run_params)

Waiting for SQL query to complete (0 hours 0 minutes and a second)
Waiting for SQL query to complete (0 hours 0 minutes and 12 seconds)
Run finished successfully


In [23]:
response

SQLQuery(session=<rhino_health.lib.rhino_session.RhinoSession object at 0x10aa3e040>, creator_uid='e50f2461-3d7a-4fe0-b457-b2fe45d05532', created_at='Mar 04, 2024 07:57:40PM', project_uid='1752fb63-c36c-4221-85a4-f979c5e921bb', workgroup_uid='97d7456b-baef-4d88-bfb1-afd4975eaff2', sql_query='SELECT\n   omr.subject_id,\n   omr.chartdate,\n   omr.result_name,\n   max(omr.result_value) as result\nFROM mimiciv_hosp.omr omr\nLEFT JOIN mimiciv_cxr.study_list study\nON omr.subject_id =study.subject_id\nWHERE study.study_id in (57375967,50771383,54205396,53186264,54684191,50531538,59965534,51029426,50336039,51967845,53970869,54898709,57084339,58117097,58509443,58555910,58733084,59289932,51449744,55590752,56617354,57652741,50230446,50252971,50702026,50789010,51737583,54855307,56421164,58996402,59565087,52358194,51732447,53447201,54136122,58882809,54076811,53022275,58261299,57661212,52821744,53831730,56167317,57754443,50548939,55758528,58974095,51613820,58898689,55328702,52654671,53468612,559283

### Importing chest x-rays from a PACS system into my Rhino client
Next, we'll import chest x-rays onto my Rhino client so that we can conduct a computer vision experiment in the following steps:.

**To enable a friction-less guided sandbox experience, Rhino staff have uploaded DICOM data into the project for you.** If you are interested in learning more about how data can be imported from your local computing environment into the Rhino Federated Computing Platform, please refer to this section of our documentation [here](https://docs.rhinohealth.com/hc/en-us/articles/12385912890653-Adding-Data-to-your-Rhino-Federated-Computing-Platform-Client).

The data has been loaded in the `/rhino_data/image/dicom` path in the Rhino client. In addition, a file that provides metadata to associate the DICOM studies with the EHR data has been imported ('/rhino_data/image/metadata/aidev_cohort.csv').

In [24]:
dicom_path='/rhino_data/aidev/dicom'
metadata_file = '/rhino_data/aidev/aidev_cohort.csv'

cohort_creation_params = DatasetCreateInput(
    session=session,
    name="mimic_cxr_dev",
    description="mimic_cxr_dev",
    project=project_uid,
    workgroup=workgroup_uid,
    data_schema=None,
    image_filesystem_location=dicom_path,
    csv_filesystem_location = metadata_file,
    is_data_deidentified=True,
    method="filesystem",
)

ai_developer_image_cohort = session.dataset.add_dataset(cohort_creation_params)
print(f"Created new cohort '{ai_developer_image_cohort.name}' with uid '{ai_developer_image_cohort.uid}'")

Created new cohort 'mimic_cxr_dev' with uid 'a45aabb1-bc58-4233-a2f1-65dc1a151178'


<!-- ```python
dicom_path = "/rhino_data/image/dicom"
metadata_file = "/rhino_data/image/metadata/aidev_cohort.csv"

cohort_creation_params = CohortCreateInput(
    name="mimic_cxr_dev",
    description="mimic_cxr_dev",
    project_uid=project_uid, 
    workgroup_uid=workgroup_uid,
    image_filesystem_location=dicom_path,
    csv_filesystem_location = metadata_file,
    is_data_deidentified=True,
    method="filesystem",
)

ai_developer_image_cohort = session.cohort.add_cohort(cohort_creation_params)
print(f"Created new cohort '{ai_developer_image_cohort.name}' with uid '{ai_developer_image_cohort.uid}'")
``` -->

### What you'll see in the Rhino UI:
Once all three queries have been executed, you should see three cohorts in the user interface:
![Mimic Cohorts in the FCP](https://drive.google.com/uc?export=view&id=1f6BaoyG3-4kmw4vUSr_WGNPq5_ZD3awC)

### Where is my data in the Rhino client?  
Once data is uploaded, it'll reside in your designated Rhino client. While the Rhino Federated Computing Platform eliminates the need for the user to know the path of the data (enabling users just to refer to 'cohorts' it'll reside in the `/rhino_data/image/dicom` folder. 
![image info](https://drive.google.com/uc?export=view&id=1oHvJhcU_My3cNWg8-amjStj8sA2t_iwQ)

To learn more about working with DICOM data on the Rhino Federated Computing Platform, please refer to our documentation [here](https://docs.rhinohealth.com/hc/en-us/articles/13136536913693-Example-1-Defining-a-Cohort-with-DICOM-Data).